In [ ]:
!nvidia-smi

Thu Feb 17 07:21:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import tensorflow as tf
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# InceptionNet

## Inception Block

In [2]:
class InceptionBlock(tf.keras.layers.Layer):
    def __init__(self, c1, c2, c3, c4):
        '''
        Input
        c1(int): number of filters for 1 x 1 convolution layer of path 1
        c2(tuple): number of filters for 1 x 1 convolution layer and 3 x 3 convolution layer of path 2
        c3(tuple): number of filters for 1 x 1 convolution layer and 5 x 4 convolution layer of path 3
        c4(int): number of filters for 1 x 1 convolution layer of path 4
        '''
        super(InceptionBlock, self).__init__()

        self.p1_1 = tf.keras.layers.Conv2D(filters=c1, kernel_size=1,
                                           padding='same', activation='relu')
        
        self.p2_1 = tf.keras.layers.Conv2D(filters=c2[0], kernel_size=1,
                                           activation='relu')
        self.p2_2 = tf.keras.layers.Conv2D(filters=c2[1], kernel_size=3,
                                           padding='same', activation='relu')
        
        self.p3_1 = tf.keras.layers.Conv2D(filters=c3[0], kernel_size=1,
                                           activation='relu')
        self.p3_2 = tf.keras.layers.Conv2D(filters=c3[1], kernel_size=5,
                                           padding='same', activation='relu')
        
        self.p4_1 = tf.keras.layers.MaxPool2D(pool_size=3, strides=1, padding='same')
        self.p4_2 = tf.keras.layers.Conv2D(c4, 1, activation='relu')

    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))

        return tf.keras.layers.Concatenate()([p1, p2, p3, p4])


In [3]:
X = tf.random.uniform((1, 28, 28, 192))
block = InceptionBlock(64, (96, 128), (16, 32), 32)
block(X).shape

TensorShape([1, 28, 28, 256])

## InceptionNet(GooGLENet)

In [8]:
class InceptionNet(tf.keras.Model):
    def __init__(self, num_classes):
        '''
        Input
        num_classes(int): 
        '''
        super(InceptionNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=7,
                                            strides=2, padding='same',
                                            activation='relu')
        self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2,
                                                  padding='same')
        self.conv2 = tf.keras.layers.Conv2D(filters=192, kernel_size=3, 
                                            strides=1, padding='same',
                                            activation='relu')
        self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2,
                                                  padding='same')
        
        self.inception3a = InceptionBlock(64, (96, 128), (16, 32), 32)
        self.inception3b = InceptionBlock(128, (128, 192), (32, 96), 64) 
        self.maxpool3 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2,
                                                  padding='same')
        
        self.inception4a = InceptionBlock(192, (96, 208), (16, 48), 64) 
        self.inception4b = InceptionBlock(160, (112, 224), (24, 64), 64) 
        self.inception4c = InceptionBlock(128, (128, 256), (24, 64), 64) 
        self.inception4d = InceptionBlock(112, (144, 288), (32, 64), 64) 
        self.inception4e = InceptionBlock(256, (160, 320), (32, 128), 128)
        self.maxpool4 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=2, 
                                                  padding='same')
        
        self.inception5a = InceptionBlock(256, (160, 320), (32, 128), 128)
        self.inception5b = InceptionBlock(384, (192, 384), (48, 128), 128) 
        self.avgpool = tf.keras.layers.AveragePooling2D(pool_size=(7, 7), strides=1)
        
        self.dropout = tf.keras.layers.Dropout(rate=0.4)
        self.dense = tf.keras.layers.Dense(units=num_classes, activation='softmax')

    def call(self, X):
        X = self.conv1(X)
        X = self.maxpool1(X)
        X = self.conv2(X)
        X = self.maxpool2(X)
        X = self.inception3a(X)
        X = self.inception3b(X)
        X = self.maxpool3(X)
        X = self.inception4a(X)
        X = self.inception4b(X)
        X = self.inception4c(X)
        X = self.inception4d(X)
        X = self.inception4e(X)
        X = self.maxpool4(X)
        X = self.inception5a(X)
        X = self.inception5b(X)
        X = self.avgpool(X)
        X = self.dropout(X)
        return self.dense(X)
    

In [13]:
model = InceptionNet(num_classes=1000)
X = tf.random.uniform((1, 224, 224, 3))
model(X)

<tf.Tensor: shape=(1, 1, 1, 1000), dtype=float32, numpy=
array([[[[0.00099327, 0.00098925, 0.0009883 , 0.00100536, 0.00099733,
          0.00098871, 0.00100068, 0.0009975 , 0.00100316, 0.00098823,
          0.0010083 , 0.00099416, 0.00100492, 0.00101271, 0.00099818,
          0.00100651, 0.00099659, 0.00099071, 0.00099774, 0.00099708,
          0.00099664, 0.00100016, 0.00099657, 0.00100114, 0.00099016,
          0.00099457, 0.00099508, 0.0009994 , 0.00100112, 0.00100383,
          0.00099607, 0.00099237, 0.00099639, 0.00101102, 0.00099551,
          0.00099269, 0.00099559, 0.00099223, 0.00100049, 0.00100251,
          0.00099825, 0.00099882, 0.00099319, 0.0009999 , 0.00099652,
          0.00100231, 0.00098861, 0.00098654, 0.00100349, 0.00100013,
          0.00100634, 0.00101662, 0.00100875, 0.00100407, 0.000995  ,
          0.00099598, 0.00099911, 0.00099602, 0.00100599, 0.00098786,
          0.001001  , 0.00099766, 0.00099572, 0.00100327, 0.001003  ,
          0.00100224, 0.00100246,

In [14]:
model.summary()

Model: "inception_net_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_119 (Conv2D)         multiple                  9472      
                                                                 
 max_pooling2d_27 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 conv2d_120 (Conv2D)         multiple                  110784    
                                                                 
 max_pooling2d_28 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 inception_block_19 (Incepti  multiple                 163696    
 onBlock)                                                        
                                                   

# Download and pre-processing dataset

In [ ]:
!gdown --id 1RL0T7Rg4XqQNRCkjfnLo4goOJQ7XZro9
!unzip cats_and_dogs_filtered.zip

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, 
                                   width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True, vertical_flip=True, 
                                   fill_mode='nearest')

train_dataset = train_datagen.flow_from_directory(directory='/content/cats_and_dogs_filtered/train', batch_size=20,
                                                  class_mode='categorical', target_size=(224, 224))

val_datagen = ImageDataGenerator(rescale=1./255)

val_dataset = val_datagen.flow_from_directory(directory='/content/cats_and_dogs_filtered/validation', batch_size=20, 
                                              class_mode='categorical', target_size=(224, 224))

In [17]:
model = InceptionNet(num_classes=2)
model.build(input_shape=(None, 224, 224, 3))

In [ ]:
EPOCHS = 20
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()
batch_size = 20

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_function(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits) 
    return loss_value 

@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

for epoch in range(EPOCHS):

    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train) 
        if step % 100 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))  

    train_acc = train_acc_metric.result()
    print("Training acc at step: %.4f" % (float(train_acc),))
    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()

    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))                         